In [1]:
import pandas as pd
import numpy as np

# データ取得

In [2]:
data = pd.read_csv('data/train.csv')

In [3]:
data = data[['jap_question_1_wakati', 'jap_question_2_wakati', 'is_duplicate']]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 3 columns):
jap_question_1_wakati    24000 non-null object
jap_question_2_wakati    24000 non-null object
is_duplicate             24000 non-null int64
dtypes: int64(1), object(2)
memory usage: 562.6+ KB


In [5]:
from sklearn.model_selection import train_test_split
_, data = train_test_split(data, shuffle=True, random_state=123)

# tf-idfの取得

In [6]:
data_all = pd.concat([data['jap_question_1_wakati'], data['jap_question_1_wakati']], axis=0
                    ).to_numpy()

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
vec_tfidf = TfidfVectorizer()

vec_tfidf.fit(data_all)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [9]:
data1 = vec_tfidf.transform(data['jap_question_1_wakati'].to_numpy())
data2 = vec_tfidf.transform(data['jap_question_2_wakati'].to_numpy())

assert data1.shape == data2.shape and data1.shape[0] == len(data)

# calculate cosine similarity
tf-idfのcosine similarityが大きさでquestion pairsか否か判定する

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
cos_dist = cosine_similarity(data1, data2)

In [12]:
cos_dist.shape

(6000, 6000)

In [13]:
cos_dist_diag = np.diag(cos_dist)
# cos_dist_diag = np.reshape(cos_dist_diag, (cos_dist_diag.shape[0], 1))

In [14]:
cos_dist_diag.shape

(6000,)

In [15]:
cos = pd.Series(cos_dist_diag)

In [16]:
data['cos_sim']= cos

# evaluate
色々な範囲でしきい値を設けて正解率を算出

In [17]:
data = data.assign(
    over80=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.8 else 0),
    over75=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.75 else 0),
    over70=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.7 else 0),
    over65=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.65 else 0),
    over60=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.6 else 0),
    over55=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.55 else 0),
    over50=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.5 else 0),
    over45=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.45 else 0),
    over40=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.4 else 0),
    over35=lambda df: df['cos_sim'].apply(lambda x: 1 if x >= 0.35 else 0),
)

data['rand'] = pd.Series(np.random.randint(0, 2, len(data)), index=data.index )

In [18]:
num = len(data)


acc = len(data[data['is_duplicate'] == data['rand']]) / num
print('random acc: ', acc)

for i in range(80, 30, -5):
    acc = len(data[data['is_duplicate'] == data['over{}'.format(i)]]) / num
    print('over{} acc: '.format(i), acc)

random acc:  0.49483333333333335
over80 acc:  0.6216666666666667
over75 acc:  0.6155
over70 acc:  0.6128333333333333
over65 acc:  0.61
over60 acc:  0.607
over55 acc:  0.6031666666666666
over50 acc:  0.5993333333333334
over45 acc:  0.5951666666666666
over40 acc:  0.5943333333333334
over35 acc:  0.5918333333333333
